### Yolov8n

In [1]:

from ultralytics import YOLO
from pathlib import Path
import matplotlib.pyplot as plt
import pandas as pd
import os
import shutil
import time

# Configs
dataset_dir = r"C:\Users\PS_22_2\Desktop\dataset_new_copy"
data_yaml = r"D:\DeepSORT_ML2025\yolo_project\data.yaml"
output_dir = r"D:\DeepSORT_ML2025\yolo_project"
timestamp = pd.Timestamp.now().strftime("%Y%m%d_%H%M%S")
run_name = f"marmoset_yolo_{timestamp}"

epochs = 250
batch_size = 8
img_size = 640


model = YOLO('yolov8n.pt')

#Train the model
results = model.train(
    data=data_yaml,
    epochs=epochs,
    batch=batch_size,
    imgsz=img_size,
    project=output_dir,
    name=run_name,
    exist_ok=True
)

# Find latest training run folder
train_runs = sorted(Path(output_dir).glob('marmoset_yolo_*'), key=os.path.getmtime)
if not train_runs:
    raise FileNotFoundError("No training runs found!")
latest_run = train_runs[-1]


def plot_live(df, save_path):
    plt.figure(figsize=(10, 6))

    if 'train/box_loss' in df.columns and 'metrics/val/box_loss' in df.columns:
        plt.plot(df['train/box_loss'], label='Train Box Loss')
        plt.plot(df['metrics/val/box_loss'], label='Val Box Loss')

    if 'train/cls_loss' in df.columns and 'metrics/val/cls_loss' in df.columns:
        plt.plot(df['train/cls_loss'], label='Train Cls Loss')
        plt.plot(df['metrics/val/cls_loss'], label='Val Cls Loss')

    if 'train/dfl_loss' in df.columns and 'metrics/val/dfl_loss' in df.columns:
        plt.plot(df['train/dfl_loss'], label='Train DFL Loss')
        plt.plot(df['metrics/val/dfl_loss'], label='Val DFL Loss')

    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('YOLOv8 Training vs Validation Losses')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(save_path)
    plt.close()


metrics_file = latest_run / 'results.csv'
while not metrics_file.exists():
    time.sleep(1)

epochs_so_far = 0
while epochs_so_far < epochs:
    df = pd.read_csv(metrics_file)
    if len(df) > epochs_so_far:
        plot_save_path = latest_run / 'results.png'
        plot_live(df, plot_save_path)
        epochs_so_far = len(df)
    time.sleep(1)


df = pd.read_csv(metrics_file)
if 'metrics/mAP_0.5' in df.columns and 'metrics/mAP_0.5:0.95' in df.columns:
    plt.figure(figsize=(12, 6))
    plt.plot(df['metrics/mAP_0.5'], label='mAP@0.5')
    plt.plot(df['metrics/mAP_0.5:0.95'], label='mAP@0.5:0.95')
    plt.xlabel('Epoch')
    plt.ylabel('mAP')
    plt.title('Validation mAP over Epochs')
    plt.legend()
    plt.grid(True)
    mAP_plot_path = latest_run / 'mAP_plot.png'
    plt.savefig(mAP_plot_path)
    plt.close()
    print(f"\n mAP plot saved to: {mAP_plot_path}")
else:
    print(" mAP metrics not found in results.csv.")


best_model = latest_run / 'weights' / 'best.pt'
custom_save_path = Path(output_dir) / f"marmoset_yolo_best_{timestamp}.pt"

if best_model.exists():
    shutil.copy(best_model, custom_save_path)
    print(f"\n Best model saved as: {custom_save_path}")
else:
    print("\n Best model file not found!")


C:\Users\PS_22_2\AppData\Roaming\Python\Python310\site-packages\pandas\core\arrays\masked.py:61: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


100%|██████████| 6.25M/6.25M [00:00<00:00, 20.7MB/s]


New https://pypi.org/project/ultralytics/8.3.170 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.156  Python-3.10.9 torch-2.5.1+cu121 CUDA:0 (NVIDIA RTX A2000, 6138MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=D:\DeepSORT_ML2025\yolo_project\data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=250, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=marmoset_yolo_20250728_185307, nbs=

100%|██████████| 5.35M/5.35M [00:00<00:00, 21.9MB/s]


AMP: checks passed 
train: Fast image access  (ping: 0.10.0 ms, read: 1306.0468.8 MB/s, size: 944.7 KB)


train: Scanning C:\Users\PS_22_2\Desktop\dataset_new_copy\labels\train.cache... 751 images, 0 backgrounds, 43 corrupt: 100%|██████████| 751/751 [00:00<?, ?it/s]

train: C:\Users\PS_22_2\Desktop\dataset_new_copy\images\train\LL_rew3_cam11_frame_0147.jpg: ignoring corrupt image/label: Label class 1 exceeds dataset class count 1. Possible class labels are 0-0
train: C:\Users\PS_22_2\Desktop\dataset_new_copy\images\train\LL_rew3_cam11_frame_0148.jpg: ignoring corrupt image/label: Label class 1 exceeds dataset class count 1. Possible class labels are 0-0
train: C:\Users\PS_22_2\Desktop\dataset_new_copy\images\train\LL_rew3_cam11_frame_0151.jpg: ignoring corrupt image/label: Label class 1 exceeds dataset class count 1. Possible class labels are 0-0
train: C:\Users\PS_22_2\Desktop\dataset_new_copy\images\train\LL_rew3_cam11_frame_0152.jpg: ignoring corrupt image/label: Label class 1 exceeds dataset class count 1. Possible class labels are 0-0
train: C:\Users\PS_22_2\Desktop\dataset_new_copy\images\train\LL_rew3_cam11_frame_0153.jpg: ignoring corrupt image/label: Label class 1 exceeds dataset class count 1. Possible class labels are 0-0
train: C:\Users

KeyboardInterrupt: 

In [ ]:
# === SETUP ===
from ultralytics import YOLO
from pathlib import Path
import matplotlib.pyplot as plt
import pandas as pd
import os
import shutil
import time

# === Configs ===
dataset_dir = r"C:\Users\PS_22_2\Desktop\dataset"
data_yaml = r"D:\DeepSORT_ML2025\yolo_project\data.yaml"
output_dir = r"D:\DeepSORT_ML2025\yolo_project"
timestamp = pd.Timestamp.now().strftime("%Y%m%d_%H%M%S")
run_name = f"marmoset_yolo_{timestamp}"

epochs = 250
batch_size = 8
img_size = 640

# === Create YOLO model ===
model = YOLO('yolov8n.pt')

# === Train the model ===
results = model.train(
    data=data_yaml,
    epochs=epochs,
    batch=batch_size,
    imgsz=img_size,
    project=output_dir,
    name=run_name,
    exist_ok=True
)

# === Find latest training run folder ===
train_runs = sorted(Path(output_dir).glob('marmoset_yolo_*'), key=os.path.getmtime)
if not train_runs:
    raise FileNotFoundError("No training runs found!")
latest_run = train_runs[-1]

# === Plotting Function ===
def plot_live(df, save_path):
    plt.figure(figsize=(10, 6))

    # Plot all available losses if present
    if 'train/box_loss' in df.columns and 'metrics/val/box_loss' in df.columns:
        plt.plot(df['train/box_loss'], label='Train Box Loss')
        plt.plot(df['metrics/val/box_loss'], label='Val Box Loss')

    if 'train/cls_loss' in df.columns and 'metrics/val/cls_loss' in df.columns:
        plt.plot(df['train/cls_loss'], label='Train Cls Loss')
        plt.plot(df['metrics/val/cls_loss'], label='Val Cls Loss')

    if 'train/dfl_loss' in df.columns and 'metrics/val/dfl_loss' in df.columns:
        plt.plot(df['train/dfl_loss'], label='Train DFL Loss')
        plt.plot(df['metrics/val/dfl_loss'], label='Val DFL Loss')

    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('YOLOv8 Training vs Validation Losses')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(save_path)
    plt.close()
    print(f"[INFO] Saved loss plot to {save_path}")

# === Live Plot and Save Loop ===
metrics_file = latest_run / 'results.csv'
while not metrics_file.exists():
    time.sleep(1)

epochs_so_far = 0
plot_save_path = latest_run / 'results.png'

while epochs_so_far < epochs:
    df = pd.read_csv(metrics_file)
    if len(df) > epochs_so_far:
        plot_live(df, plot_save_path)
        epochs_so_far = len(df)
    time.sleep(1)

# === Plot final validation mAP ===
df = pd.read_csv(metrics_file)
if 'metrics/mAP_0.5' in df.columns and 'metrics/mAP_0.5:0.95' in df.columns:
    plt.figure(figsize=(12, 6))
    plt.plot(df['metrics/mAP_0.5'], label='mAP@0.5')
    plt.plot(df['metrics/mAP_0.5:0.95'], label='mAP@0.5:0.95')
    plt.xlabel('Epoch')
    plt.ylabel('mAP')
    plt.title('Validation mAP over Epochs')
    plt.legend()
    plt.grid(True)
    mAP_plot_path = latest_run / 'mAP_plot.png'
    plt.savefig(mAP_plot_path)
    plt.close()
    print(f"\n mAP plot saved to: {mAP_plot_path}")
else:
    print(" mAP metrics not found in results.csv.")

# === Save best model with timestamp ===
best_model = latest_run / 'weights' / 'best.pt'
custom_save_path = Path(output_dir) / f"marmoset_yolo_best_{timestamp}.pt"

if best_model.exists():
    shutil.copy(best_model, custom_save_path)
    print(f"\n Best model saved as: {custom_save_path}")
else:
    print("\n Best model file not found!")


Ultralytics 8.3.156  Python-3.10.9 torch-2.5.1+cu121 CUDA:0 (NVIDIA RTX A2000, 6138MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=D:\DeepSORT_ML2025\yolo_project\data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=250, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=marmoset_yolo_20250617_171101, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspecti

train: Scanning C:\Users\PS_22_2\Desktop\dataset\labels\train.cache... 519 images, 0 backgrounds, 46 corrupt: 100%|██████████| 519/519 [00:00<?, ?it/s]

train: C:\Users\PS_22_2\Desktop\dataset\images\train\LL_rew3_cam11_frame_0146.jpg: ignoring corrupt image/label: Label class 1 exceeds dataset class count 1. Possible class labels are 0-0
train: C:\Users\PS_22_2\Desktop\dataset\images\train\LL_rew3_cam11_frame_0148.jpg: ignoring corrupt image/label: Label class 1 exceeds dataset class count 1. Possible class labels are 0-0
train: C:\Users\PS_22_2\Desktop\dataset\images\train\LL_rew3_cam11_frame_0149.jpg: ignoring corrupt image/label: Label class 1 exceeds dataset class count 1. Possible class labels are 0-0
train: C:\Users\PS_22_2\Desktop\dataset\images\train\LL_rew3_cam11_frame_0151.jpg: ignoring corrupt image/label: Label class 1 exceeds dataset class count 1. Possible class labels are 0-0
train: C:\Users\PS_22_2\Desktop\dataset\images\train\LL_rew3_cam11_frame_0152.jpg: ignoring corrupt image/label: Label class 1 exceeds dataset class count 1. Possible class labels are 0-0
train: C:\Users\PS_22_2\Desktop\dataset\images\train\LL_rew3

val: Fast image access  (ping: 0.00.0 ms, read: 1589.9677.4 MB/s, size: 830.3 KB)


val: Scanning C:\Users\PS_22_2\Desktop\dataset\labels\val.cache... 130 images, 0 backgrounds, 16 corrupt: 100%|██████████| 130/130 [00:00<?, ?it/s]

...


250 epochs completed in 0.667 hours.
Optimizer stripped from D:\DeepSORT_ML2025\yolo_project\marmoset_yolo_20250617_171101\weights\last.pt, 6.3MB
Optimizer stripped from D:\DeepSORT_ML2025\yolo_project\marmoset_yolo_20250617_171101\weights\best.pt, 6.3MB

Validating D:\DeepSORT_ML2025\yolo_project\marmoset_yolo_20250617_171101\weights\best.pt...
Ultralytics 8.3.156  Python-3.10.9 torch-2.5.1+cu121 CUDA:0 (NVIDIA RTX A2000, 6138MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:01<00:00,  5.52it/s]


                   all        114        158      0.955      0.937      0.971      0.789
Speed: 0.3ms preprocess, 2.5ms inference, 0.0ms loss, 2.0ms postprocess per image
Results saved to D:\DeepSORT_ML2025\yolo_project\marmoset_yolo_20250617_171101


C:\Users\PS_22_2\AppData\Local\Temp\ipykernel_18484\2947163172.py:61: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  plt.legend()


[INFO] Saved loss plot to D:\DeepSORT_ML2025\yolo_project\marmoset_yolo_20250617_171101\results.png
⚠️ mAP metrics not found in results.csv.

✅ Best model saved as: D:\DeepSORT_ML2025\yolo_project\marmoset_yolo_best_20250617_171101.pt


### Yolov8m

In [ ]:

from ultralytics import YOLO
from pathlib import Path
import matplotlib.pyplot as plt
import pandas as pd
import os
import shutil
import time

dataset_dir = r"C:\Users\PS_22_2\Desktop\dataset"
data_yaml = r"D:\DeepSORT_ML2025\yolo_project\data.yaml"
output_dir = Path(r"D:\DeepSORT_ML2025\yolo_project")
timestamp = pd.Timestamp.now().strftime("%Y%m%d_%H%M%S")
run_name = f"marmoset_yolo_{timestamp}"

# Model and training settings
epochs = 250
batch_size = 8
img_size = 640
model_choice = 'yolov8m.pt' 
early_stop_patience = 20


model = YOLO(model_choice)

# Train the model 
results = model.train(
    data=data_yaml,
    epochs=epochs,
    batch=batch_size,
    imgsz=img_size,
    project=str(output_dir),
    name=run_name,
    exist_ok=True,
    patience=early_stop_patience,
    verbose=True
)

#Locate latest training run folder
train_runs = sorted(output_dir.glob('marmoset_yolo_*'), key=os.path.getmtime)
if not train_runs:
    raise FileNotFoundError("No training runs found!")
latest_run = train_runs[-1]


def plot_live(df, save_path):
    plt.figure(figsize=(12, 6))

    if 'train/box_loss' in df.columns and 'metrics/val/box_loss' in df.columns:
        plt.plot(df['train/box_loss'], label='Train Box Loss')
        plt.plot(df['metrics/val/box_loss'], label='Val Box Loss')
        plt.plot(df['metrics/val/box_loss'] - df['train/box_loss'], label='Box Loss Gap (Overfit)', linestyle='dotted', color='red')

    if 'train/cls_loss' in df.columns and 'metrics/val/cls_loss' in df.columns:
        plt.plot(df['train/cls_loss'], label='Train Cls Loss')
        plt.plot(df['metrics/val/cls_loss'], label='Val Cls Loss')

    if 'train/dfl_loss' in df.columns and 'metrics/val/dfl_loss' in df.columns:
        plt.plot(df['train/dfl_loss'], label='Train DFL Loss')
        plt.plot(df['metrics/val/dfl_loss'], label='Val DFL Loss')

    if 'metrics/mAP_0.5' in df.columns:
        ax2 = plt.gca().twinx()
        ax2.plot(df['metrics/mAP_0.5'], label='Val mAP@0.5', color='green', linestyle='--')
        ax2.set_ylabel('mAP@0.5', color='green')

    plt.xlabel('Epoch')
    plt.title('Training & Validation Losses + mAP')
    plt.legend(loc='upper right')
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(save_path)
    plt.close()
    print(f"[INFO] Saved loss + mAP plot to {save_path}")


metrics_file = latest_run / 'results.csv'
while not metrics_file.exists():
    time.sleep(1)

epochs_so_far = 0
plot_save_path = latest_run / 'results_live.png'

while epochs_so_far < epochs:
    df = pd.read_csv(metrics_file)
    if len(df) > epochs_so_far:
        plot_live(df, plot_save_path)
        epochs_so_far = len(df)
    time.sleep(1)


df = pd.read_csv(metrics_file)
if 'metrics/mAP_0.5' in df.columns and 'metrics/mAP_0.5:0.95' in df.columns:
    plt.figure(figsize=(12, 6))
    plt.plot(df['metrics/mAP_0.5'], label='mAP@0.5')
    plt.plot(df['metrics/mAP_0.5:0.95'], label='mAP@0.5:0.95')
    plt.xlabel('Epoch')
    plt.ylabel('mAP')
    plt.title('Validation mAP Over Epochs')
    plt.legend()
    plt.grid(True)
    mAP_plot_path = latest_run / 'mAP_plot.png'
    plt.savefig(mAP_plot_path)
    plt.close()
    print(f"Final mAP plot saved to: {mAP_plot_path}")
else:
    print("mAP metrics not found in results.csv.")


best_model = latest_run / 'weights' / 'best.pt'
custom_save_path = output_dir / f"marmoset_yolo_best_{timestamp}.pt"
if best_model.exists():
    shutil.copy(best_model, custom_save_path)
    print(f"Best model saved as: {custom_save_path}")
else:
    print("Best model file not found!")


summary_file = latest_run / "training_summary.txt"
with open(summary_file, "w") as f:
    f.write(f"Run: {run_name}\n")
    f.write(f"Model: {model_choice}\n")
    f.write(f"Epochs: {len(df)} / {epochs}\n")
    f.write(f"Batch size: {batch_size}\n")
    f.write(f"Image size: {img_size}\n\n")

    if 'metrics/mAP_0.5' in df.columns:
        best_epoch = df['metrics/mAP_0.5'].idxmax()
        f.write(f"Best Epoch: {best_epoch}\n")
        f.write(f"Best mAP@0.5: {df['metrics/mAP_0.5'].max():.4f}\n")
        f.write(f"Best mAP@0.5:0.95: {df['metrics/mAP_0.5:0.95'].max():.4f}\n")
    else:
        f.write("mAP metrics not available.\n")
print(f"Training summary saved at: {summary_file}")

# === Optional: save validation predictions as images ===
try:
    print("[INFO] Generating final validation predictions...")
    model.val(data=data_yaml, save=True, imgsz=img_size, batch=batch_size)
except Exception as e:
    print(f"Could not save validation predictions: {e}")


C:\Users\PS_22_2\AppData\Roaming\Python\Python310\site-packages\pandas\core\arrays\masked.py:61: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


100%|██████████| 49.7M/49.7M [00:03<00:00, 13.7MB/s]


Ultralytics 8.3.156  Python-3.10.9 torch-2.5.1+cu121 CUDA:0 (NVIDIA RTX A2000, 6138MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=D:\DeepSORT_ML2025\yolo_project\data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=250, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8m.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=marmoset_yolo_20250618_174155, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=20, perspectiv

train: Scanning C:\Users\PS_22_2\Desktop\dataset\labels\train... 608 images, 0 backgrounds, 52 corrupt: 100%|██████████| 608/608 [00:03<00:00, 191.77it/s]

train: C:\Users\PS_22_2\Desktop\dataset\images\train\LL_rew3_cam11_frame_0146.jpg: ignoring corrupt image/label: Label class 1 exceeds dataset class count 1. Possible class labels are 0-0
train: C:\Users\PS_22_2\Desktop\dataset\images\train\LL_rew3_cam11_frame_0148.jpg: ignoring corrupt image/label: Label class 1 exceeds dataset class count 1. Possible class labels are 0-0
train: C:\Users\PS_22_2\Desktop\dataset\images\train\LL_rew3_cam11_frame_0149.jpg: ignoring corrupt image/label: Label class 1 exceeds dataset class count 1. Possible class labels are 0-0
train: C:\Users\PS_22_2\Desktop\dataset\images\train\LL_rew3_cam11_frame_0150.jpg: ignoring corrupt image/label: Label class 1 exceeds dataset class count 1. Possible class labels are 0-0
train: C:\Users\PS_22_2\Desktop\dataset\images\train\LL_rew3_cam11_frame_0151.jpg: ignoring corrupt image/label: Label class 1 exceeds dataset class count 1. Possible class labels are 0-0
train: C:\Users\PS_22_2\Desktop\dataset\images\train\LL_rew3

...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:02<00:00,  4.28it/s]


                   all        142        203      0.933      0.852      0.947      0.747
Speed: 0.3ms preprocess, 7.8ms inference, 0.0ms loss, 2.7ms postprocess per image
Results saved to D:\DeepSORT_ML2025\yolo_project\marmoset_yolo_20250618_174155


C:\Users\PS_22_2\AppData\Local\Temp\ipykernel_24712\203986238.py:70: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  plt.legend(loc='upper right')


[INFO] Saved loss + mAP plot to D:\DeepSORT_ML2025\yolo_project\marmoset_yolo_20250618_174155\results_live.png


KeyboardInterrupt: 

In [1]:

from ultralytics import YOLO
from pathlib import Path
import matplotlib.pyplot as plt
import pandas as pd
import os
import shutil
import time


dataset_dir = r"C:\Users\PS_22_2\Desktop\dataset_ne_copy"
data_yaml = r"D:\DeepSORT_ML2025\yolo_project\data.yaml"
output_dir = Path(r"D:\DeepSORT_ML2025\yolo_project")
timestamp = pd.Timestamp.now().strftime("%Y%m%d_%H%M%S")
run_name = f"marmoset_yolo_{timestamp}"


epochs = 250
batch_size = 8
img_size = 640
model_choice = 'yolov8m.pt'  
early_stop_patience = 20


model = YOLO(model_choice)


results = model.train(
    data=data_yaml,
    epochs=epochs,
    batch=batch_size,
    imgsz=img_size,
    project=str(output_dir),
    name=run_name,
    exist_ok=True,
    patience=early_stop_patience,
    verbose=True
)


train_runs = sorted(output_dir.glob('marmoset_yolo_*'), key=os.path.getmtime)
if not train_runs:
    raise FileNotFoundError("No training runs found!")
latest_run = train_runs[-1]


def plot_live(df, save_path):
    plt.figure(figsize=(12, 6))

    if 'train/box_loss' in df.columns and 'metrics/val/box_loss' in df.columns:
        plt.plot(df['train/box_loss'], label='Train Box Loss')
        plt.plot(df['metrics/val/box_loss'], label='Val Box Loss')
        if 'metrics/test/box_loss' in df.columns:  # <--- added
            plt.plot(df['metrics/test/box_loss'], label='Test Box Loss', linestyle='--')  # <--- added
        plt.plot(df['metrics/val/box_loss'] - df['train/box_loss'], label='Box Loss Gap (Overfit)', linestyle='dotted', color='red')

    if 'train/cls_loss' in df.columns and 'metrics/val/cls_loss' in df.columns:
        plt.plot(df['train/cls_loss'], label='Train Cls Loss')
        plt.plot(df['metrics/val/cls_loss'], label='Val Cls Loss')
        if 'metrics/test/cls_loss' in df.columns:  # <--- added
            plt.plot(df['metrics/test/cls_loss'], label='Test Cls Loss', linestyle='--')  # <--- added

    if 'train/dfl_loss' in df.columns and 'metrics/val/dfl_loss' in df.columns:
        plt.plot(df['train/dfl_loss'], label='Train DFL Loss')
        plt.plot(df['metrics/val/dfl_loss'], label='Val DFL Loss')
        if 'metrics/test/dfl_loss' in df.columns:  # <--- added
            plt.plot(df['metrics/test/dfl_loss'], label='Test DFL Loss', linestyle='--')  # <--- added

    if 'metrics/mAP_0.5' in df.columns:
        ax2 = plt.gca().twinx()
        ax2.plot(df['metrics/mAP_0.5'], label='Val mAP@0.5', color='green', linestyle='--')
        if 'metrics/test/mAP_0.5' in df.columns:  # <--- added
            ax2.plot(df['metrics/test/mAP_0.5'], label='Test mAP@0.5', color='blue', linestyle='--')  # <--- added
        ax2.set_ylabel('mAP@0.5')

    plt.xlabel('Epoch')
    plt.title('Training/Val/Test Losses + mAP')
    plt.legend(loc='upper right')
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(save_path)
    plt.close()
    print(f"[INFO] Saved loss + mAP plot to {save_path}")


metrics_file = latest_run / 'results.csv'
while not metrics_file.exists():
    time.sleep(1)

epochs_so_far = 0
plot_save_path = latest_run / 'results_live.png'

while epochs_so_far < epochs:
    df = pd.read_csv(metrics_file)
    if len(df) > epochs_so_far:
        plot_live(df, plot_save_path)
        epochs_so_far = len(df)
    time.sleep(1)


df = pd.read_csv(metrics_file)
if 'metrics/mAP_0.5' in df.columns and 'metrics/mAP_0.5:0.95' in df.columns:
    plt.figure(figsize=(12, 6))
    plt.plot(df['metrics/mAP_0.5'], label='Val mAP@0.5')
    plt.plot(df['metrics/mAP_0.5:0.95'], label='Val mAP@0.5:0.95')
    if 'metrics/test/mAP_0.5' in df.columns:  
        plt.plot(df['metrics/test/mAP_0.5'], label='Test mAP@0.5')  
    if 'metrics/test/mAP_0.5:0.95' in df.columns: 
        plt.plot(df['metrics/test/mAP_0.5:0.95'], label='Test mAP@0.5:0.95')  
    plt.xlabel('Epoch')
    plt.ylabel('mAP')
    plt.title('Validation & Test mAP Over Epochs')
    plt.legend()
    plt.grid(True)
    mAP_plot_path = latest_run / 'mAP_plot.png'
    plt.savefig(mAP_plot_path)
    plt.close()
    print(f"Final mAP plot saved to: {mAP_plot_path}")
else:
    print("mAP metrics not found in results.csv.")


best_model = latest_run / 'weights' / 'best.pt'
custom_save_path = output_dir / f"marmoset_yolo_best_{timestamp}.pt"
if best_model.exists():
    shutil.copy(best_model, custom_save_path)
    print(f"Best model saved as: {custom_save_path}")
else:
    print("Best model file not found!")


summary_file = latest_run / "training_summary.txt"
with open(summary_file, "w") as f:
    f.write(f"Run: {run_name}\n")
    f.write(f"Model: {model_choice}\n")
    f.write(f"Epochs: {len(df)} / {epochs}\n")
    f.write(f"Batch size: {batch_size}\n")
    f.write(f"Image size: {img_size}\n\n")

    if 'metrics/mAP_0.5' in df.columns:
        best_epoch = df['metrics/mAP_0.5'].idxmax()
        f.write(f"Best Epoch: {best_epoch}\n")
        f.write(f"Best mAP@0.5: {df['metrics/mAP_0.5'].max():.4f}\n")
        f.write(f"Best mAP@0.5:0.95: {df['metrics/mAP_0.5:0.95'].max():.4f}\n")
        if 'metrics/test/mAP_0.5' in df.columns:
            f.write(f"Best Test mAP@0.5: {df['metrics/test/mAP_0.5'].max():.4f}\n")
            f.write(f"Best Test mAP@0.5:0.95: {df['metrics/test/mAP_0.5:0.95'].max():.4f}\n")
    else:
        f.write("mAP metrics not available.\n")
print(f"Training summary saved at: {summary_file}")


try:
    print("[INFO] Generating final validation predictions...")
    model.val(data=data_yaml, save=True, imgsz=img_size, batch=batch_size)
except Exception as e:
    print(f"Could not save validation predictions: {e}")


test_images_dir = Path(dataset_dir) / "images" / "test"
test_labels_dir = Path(dataset_dir) / "labels" / "test"

if test_images_dir.exists() and test_labels_dir.exists():
    try:
        print(f"[INFO] Running test set evaluation on: {test_images_dir}")
        model.val(data=data_yaml, split='test', save=True, imgsz=img_size, batch=batch_size)
        print("Test set evaluation completed and predictions saved.")
    except Exception as e:
        print(f"Could not evaluate on test set: {e}")
else:
    print("ℹNo test set found. Skipping test evaluation.")


C:\Users\PS_22_2\AppData\Roaming\Python\Python310\site-packages\pandas\core\arrays\masked.py:61: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


New https://pypi.org/project/ultralytics/8.3.170 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.156  Python-3.10.9 torch-2.5.1+cu121 CUDA:0 (NVIDIA RTX A2000, 6138MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=D:\DeepSORT_ML2025\yolo_project\data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=250, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8m.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=marmoset_yolo_20250728_184949, nbs=

100%|██████████| 5.35M/5.35M [00:00<00:00, 10.2MB/s]


AMP: checks passed 
train: Fast image access  (ping: 0.10.0 ms, read: 683.9241.5 MB/s, size: 944.7 KB)


train: Scanning C:\Users\PS_22_2\Desktop\dataset_new_copy\labels\train.cache... 751 images, 0 backgrounds, 43 corrupt: 100%|██████████| 751/751 [00:00<?, ?it/s]

...

val: Scanning C:\Users\PS_22_2\Desktop\dataset_new_copy\labels\val.cache... 187 images, 0 backgrounds, 10 corrupt: 100%|██████████| 187/187 [00:00<?, ?it/s]

val: C:\Users\PS_22_2\Desktop\dataset_new_copy\images\val\LL_rew3_cam11_frame_0146.jpg: ignoring corrupt image/label: Label class 1 exceeds dataset class count 1. Possible class labels are 0-0
val: C:\Users\PS_22_2\Desktop\dataset_new_copy\images\val\LL_rew3_cam11_frame_0147.jpg: ignoring corrupt image/label: Label class 1 exceeds dataset class count 1. Possible class labels are 0-0
val: C:\Users\PS_22_2\Desktop\dataset_new_copy\images\val\LL_rew3_cam11_frame_0159.jpg: ignoring corrupt image/label: Label class 1 exceeds dataset class count 1. Possible class labels are 0-0
val: C:\Users\PS_22_2\Desktop\dataset_new_copy\images\val\LL_rew3_cam11_frame_0164.jpg: ignoring corrupt image/label: Label class 1 exceeds dataset class count 1. Possible class labels are 0-0
val: C:\Users\PS_22_2\Desktop\dataset_new_copy\images\val\LL_rew3_cam11_frame_0317.jpg: ignoring corrupt image/label: Label class 1 exceeds dataset class count 1. Possible class labels are 0-0
val: C:\Users\PS_22_2\Desktop\datas

Plotting labels to D:\DeepSORT_ML2025\yolo_project\marmoset_yolo_20250728_184949\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 77 weight(decay=0.0), 84 weight(decay=0.0005), 83 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to D:\DeepSORT_ML2025\yolo_project\marmoset_yolo_20250728_184949
Starting training for 250 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      1/250      3.25G      1.341      2.104      1.441         25        640:  46%|████▌     | 41/89 [00:12<00:14,  3.34it/s]


KeyboardInterrupt: 